In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#import pandas_profiling as pp
import pandas_profiling as pp
from pandas_profiling import ProfileReport
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor
from catboost import Pool

C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Admin\AppData\Local\Temp\ipykernel_7792\1674564637.py:6: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  import pandas_profiling as pp


In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
fulfilment_center = pd.read_csv('fulfilment_center_info.csv')
meal_info = pd.read_csv('meal_info.csv')


In [3]:
train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 456548 entries, 0 to 456547
Data columns (total 9 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   id                     456548 non-null  int64  
 1   week                   456548 non-null  int64  
 2   center_id              456548 non-null  int64  
 3   meal_id                456548 non-null  int64  
 4   checkout_price         456548 non-null  float64
 5   base_price             456548 non-null  float64
 6   emailer_for_promotion  456548 non-null  int64  
 7   homepage_featured      456548 non-null  int64  
 8   num_orders             456548 non-null  int64  
dtypes: float64(2), int64(7)
memory usage: 31.3 MB


In [4]:
test.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32573 entries, 0 to 32572
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     32573 non-null  int64  
 1   week                   32573 non-null  int64  
 2   center_id              32573 non-null  int64  
 3   meal_id                32573 non-null  int64  
 4   checkout_price         32573 non-null  float64
 5   base_price             32573 non-null  float64
 6   emailer_for_promotion  32573 non-null  int64  
 7   homepage_featured      32573 non-null  int64  
dtypes: float64(2), int64(6)
memory usage: 2.0 MB


In [5]:
test.head()


,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured
0,1028232,146,55,1885,158.11,159.11,0,0
1,1127204,146,55,1993,160.11,159.11,0,0
2,1212707,146,55,2539,157.14,159.14,0,0
3,1082698,146,55,2631,162.02,162.02,0,0
4,1400926,146,55,1248,163.93,163.93,0,0


In [6]:
fulfilment_center.head()


,center_id,city_code,region_code,center_type,op_area
0,11,679,56,TYPE_A,3.7
1,13,590,56,TYPE_B,6.7
2,124,590,56,TYPE_C,4.0
3,66,648,34,TYPE_A,4.1
4,94,632,34,TYPE_C,3.6


In [7]:
meal_info.head()


,meal_id,category,cuisine
0,1885,Beverages,Thai
1,1993,Beverages,Thai
2,2539,Beverages,Thai
3,1248,Beverages,Indian
4,2631,Beverages,Indian


In [8]:
meal_info.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   meal_id   51 non-null     int64 
 1   category  51 non-null     object
 2   cuisine   51 non-null     object
dtypes: int64(1), object(2)
memory usage: 1.3+ KB


In [9]:
train = pd.merge(train,fulfilment_center, on='center_id')
test = pd.merge(test,fulfilment_center, on='center_id')

train = pd.merge(train,meal_info, on='meal_id')
test = pd.merge(test,meal_info, on='meal_id')

In [10]:
train.head()


,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,city_code,region_code,center_type,op_area,category,cuisine
0,1379560,1,55,1885,136.83,152.29,0,0,177,647,56,TYPE_C,2.0,Beverages,Thai
1,1018704,2,55,1885,135.83,152.29,0,0,323,647,56,TYPE_C,2.0,Beverages,Thai
2,1196273,3,55,1885,132.92,133.92,0,0,96,647,56,TYPE_C,2.0,Beverages,Thai
3,1116527,4,55,1885,135.86,134.86,0,0,163,647,56,TYPE_C,2.0,Beverages,Thai
4,1343872,5,55,1885,146.50,147.50,0,0,215,647,56,TYPE_C,2.0,Beverages,Thai


In [11]:
train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 456548 entries, 0 to 456547
Data columns (total 15 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   id                     456548 non-null  int64  
 1   week                   456548 non-null  int64  
 2   center_id              456548 non-null  int64  
 3   meal_id                456548 non-null  int64  
 4   checkout_price         456548 non-null  float64
 5   base_price             456548 non-null  float64
 6   emailer_for_promotion  456548 non-null  int64  
 7   homepage_featured      456548 non-null  int64  
 8   num_orders             456548 non-null  int64  
 9   city_code              456548 non-null  int64  
 10  region_code            456548 non-null  int64  
 11  center_type            456548 non-null  object 
 12  op_area                456548 non-null  float64
 13  category               456548 non-null  object 
 14  cuisine                456548 non-nu

In [12]:
import pandas_profiling as pp
from pandas_profiling import ProfileReport
# Assuming 'train' is your DataFrame
profile = ProfileReport(train)

# Display the profile report directly in VSCode
profile.to_file("train_profile_report.html")


Export report to file: 100%|██████████| 1/1 [00:00<00:00, 23.45it/s]


In [13]:
sns.set(style="whitegrid")  # Optional: Set a specific style for the plot

# Create the line plot
sns.lineplot(x='week', y='num_orders', data=train)

# Add the linear regression line
sns.regplot(x='week', y='num_orders', data=train, scatter=False, color='red')

plt.show()


C:\Users\Admin\AppData\Local\Temp\ipykernel_7792\2292957768.py:9: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [14]:
# sns.set(style="whitegrid")  
# aggregated_data = train.groupby(['center_id', 'category'])['num_orders'].mean().reset_index()

# sns.scatterplot(x='center_id', y='num_orders', hue='category', data=aggregated_data, marker='o', s=100)
# plt.xticks(rotation=90)
# plt.show()


In [15]:
sns.set(style="whitegrid")  # Optional: Set a specific style for the plot

# Calculate the mean 'num_orders' for each combination of 'center_id' and 'category'
aggregated_data = train.groupby(['center_id', 'category'])['num_orders'].mean().reset_index()

# Set a custom color palette with 14 colors
colors = sns.color_palette("Set1", n_colors=14)
plt.figure(figsize=(16, 6))  # Adjust the width and height as needed

# Create the scatter plot with the custom color palette
sns.scatterplot(x='center_id', y='num_orders', hue='category', data=aggregated_data, palette=colors, marker='o', s=100)

plt.xticks(rotation=90)  # Rotate x-axis labels for better visibility

plt.show()

C:\Users\Admin\AppData\Local\Temp\ipykernel_7792\669575131.py:15: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [16]:
# sns.set(style="whitegrid")  
# aggregated_data = train.groupby(['center_id', 'category'])['num_orders'].mean().reset_index()

# sns.scatterplot(x='center_id', y='num_orders', hue='category', data=aggregated_data, marker='o', s=100)
# plt.xticks(rotation=90)
# plt.show()


In [17]:
train.describe()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,city_code,region_code,op_area
count,4.565480e+05,456548.000000,456548.000000,456548.000000,456548.000000,456548.000000,456548.000000,456548.00000,456548.000000,456548.000000,456548.000000,456548.000000
mean,1.250096e+06,74.768771,82.105796,2024.337458,332.238933,354.156627,0.081152,0.10920,261.872760,601.553399,56.614566,4.083590
std,1.443548e+05,41.524956,45.975046,547.420920,152.939723,160.715914,0.273069,0.31189,395.922798,66.195914,17.641306,1.091686
min,1.000000e+06,1.000000,10.000000,1062.000000,2.970000,55.350000,0.000000,0.00000,13.000000,456.000000,23.000000,0.900000
25%,1.124999e+06,39.000000,43.000000,1558.000000,228.950000,243.500000,0.000000,0.00000,54.000000,553.000000,34.000000,3.600000
50%,1.250184e+06,76.000000,76.000000,1993.000000,296.820000,310.460000,0.000000,0.00000,136.000000,596.000000,56.000000,4.000000
75%,1.375140e+06,111.000000,110.000000,2539.000000,445.230000,458.870000,0.000000,0.00000,324.000000,651.000000,77.000000,4.500000
max,1.499999e+06,145.000000,186.000000,2956.000000,866.270000,866.270000,1.000000,1.00000,24299.000000,713.000000,93.000000,7.000000


In [18]:

sns.set(style="whitegrid")  # Optional: Set a specific style for the plot
colors = sns.color_palette("Set1", n_colors=14)

# Group by 'region' and 'category', then calculate the sum of 'num_orders'
aggregated_data = train.groupby(['region_code', 'category'])['num_orders'].sum().reset_index()
plt.figure(figsize=(16, 6))  # Adjust the width and height as needed

# Create the bar plot
sns.barplot(x='category', y='num_orders', hue='region_code', palette=colors, data=aggregated_data)

# Rotate x-axis labels for better visibility
plt.xticks(rotation=90)

plt.show()


C:\Users\Admin\AppData\Local\Temp\ipykernel_7792\3641744637.py:14: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [19]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming you have imported necessary libraries and loaded your data
plt.figure(figsize=(16, 6))  # Adjust the width and height as needed

sns.set(style="whitegrid")  # Optional: Set a specific style for the plot
bar_color = '#43521a'

# Create the bar plot
sns.barplot(x='category', y='num_orders', data=train,color=bar_color,ci=None)

# Rotate x-axis labels for better visibility
plt.xticks(rotation=90)

plt.show()


C:\Users\Admin\AppData\Local\Temp\ipykernel_7792\4066685872.py:11: FutureWarning: 

The `ci` parameter is deprecated. Use `errorbar=None` for the same effect.

  sns.barplot(x='category', y='num_orders', data=train,color=bar_color,ci=None)
C:\Users\Admin\AppData\Local\Temp\ipykernel_7792\4066685872.py:16: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [20]:
# Load the dataset
train_data = pd.read_csv('train.csv')
fulfillment_center_data = pd.read_csv('fulfilment_center_info.csv')
meal_info_data = pd.read_csv('meal_info.csv')

# Merge data
merged_data = pd.merge(train_data, fulfillment_center_data, on='center_id')
merged_data = pd.merge(merged_data, meal_info_data, on='meal_id')

# Define numerical columns for outlier detection
numerical_columns = ['checkout_price', 'base_price', 'op_area', 'num_orders']

# Create box plots to visualize outliers
plt.figure(figsize=(10, 6))
sns.boxplot(data=merged_data[numerical_columns])
plt.title('Box Plots for Outliers')
plt.xticks(rotation=45)
plt.show()

# Calculate and display z-scores to identify outliers
z_scores = (merged_data[numerical_columns] - merged_data[numerical_columns].mean()) / merged_data[numerical_columns].std()
outliers = (z_scores > 3) | (z_scores < -3)
print("Outliers:")
print(outliers)

Outliers:
        checkout_price  base_price  op_area  num_orders
0                False       False    False       False
1                False       False    False       False
2                False       False    False       False
3                False       False    False       False
4                False       False    False       False
...                ...         ...      ...         ...
456543           False       False    False       False
456544           False       False    False       False
456545           False       False    False       False
456546           False       False    False       False
456547           False       False    False       False

[456548 rows x 4 columns]


C:\Users\Admin\AppData\Local\Temp\ipykernel_7792\3378203689.py:18: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [34]:
plt.figure(figsize=(10, 5))

# Create the box plot
sns.boxplot(x=train['num_orders'])

plt.show()

In [22]:
outlier_index = train[(train['num_orders']>15000)].index
train.drop(outlier_index,inplace = True)


#Feature Engineering

In [23]:
train['train_or_test'] = 'train'
test['train_or_test'] = 'test'

In [24]:
train['num_orders'] = np.log1p(train['num_orders'])


In [26]:
total_data = pd.concat([train, test], ignore_index=True)[train.columns]


In [31]:
total_data = total_data.sort_values(['center_id', 'meal_id', 'week']).reset_index(drop=True)
total_data['checkout_price'] = np.log1p(total_data['checkout_price'])
total_data['base_price'] = np.log1p(total_data['base_price'])
total_data['discount_on_base'] = (total_data['base_price'] - total_data['checkout_price']) / total_data['base_price']
total_data["discount_ratio"] = total_data["base_price"] / total_data["checkout_price"]
total_data['price_last_curr_diff'] = (total_data['checkout_price'].shift(1) - total_data['checkout_price']).fillna(1)/ total_data['checkout_price'].shift(1).fillna(1)
for _, r in total_data.groupby(['center_id', 'meal_id'])['week'].first().reset_index().iterrows():
    total_data.loc[(total_data['center_id']==r['center_id']) & (total_data['meal_id']==r['meal_id']) & (total_data['week']==r['week']), 'price_last_curr_diff'] = total_data[(total_data['center_id']==r['center_id']) & (total_data['meal_id']==r['meal_id']) & (total_data['week']!=r['week'])]['price_last_curr_diff'].mean()
total_data['price_last_curr_diff'] = total_data['price_last_curr_diff'].fillna(0)


In [35]:
# Features constructed from previous sales values

#Creating sales lag features
def create_sales_lag_feats(df, gpby_cols, target_col, lags):
    gpby = df.groupby(gpby_cols)
    for i in lags:
        df['_'.join([target_col, 'lag', str(i)])] = \
                gpby[target_col].shift(i).values + np.random.normal(scale=1.6, size=(len(df),))
    return df

# Creating sales exponentially weighted mean features
def create_sales_ewm_feats(df, gpby_cols, target_col, alpha=[0.9], shift=[1]):
    gpby = df.groupby(gpby_cols)
    for a in alpha:
        for s in shift:
            df['_'.join([target_col, 'lag', str(s), 'ewm', str(a)])] = \
                gpby[target_col].shift(s).ewm(alpha=a).mean().values
    return df

In [36]:
# Creating num_orders lag, rolling mean, rolling median, ohe features of the above train set
total_data = create_sales_lag_feats(total_data, gpby_cols=['center_id','meal_id'], target_col='num_orders', 
                               lags=[10,11, 12])

total_data = create_sales_ewm_feats(total_data, gpby_cols=['center_id','meal_id'], 
                               target_col='num_orders', 
                               alpha=[0.5], 
                               shift=[10,11,12,13,14,15])

In [37]:
total_data.head()


,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,city_code,...,price_last_curr_diff,num_orders_lag_10,num_orders_lag_11,num_orders_lag_12,num_orders_lag_10_ewm_0.5,num_orders_lag_11_ewm_0.5,num_orders_lag_12_ewm_0.5,num_orders_lag_13_ewm_0.5,num_orders_lag_14_ewm_0.5,num_orders_lag_15_ewm_0.5
0,1436842,1,10,1062,0.537845,0.537845,0,0,6.763885,590,...,0.000021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1205013,2,10,1062,0.538020,0.537931,0,0,6.663133,590,...,-0.000326,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1447751,3,10,1062,0.538108,0.537931,0,0,6.747587,590,...,-0.000164,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1014968,4,10,1062,0.537931,0.538020,0,0,7.092574,590,...,0.000328,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1003563,5,10,1062,0.538022,0.537845,0,0,6.865891,590,...,-0.000170,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
### Center count features ###
gdf = total_data.groupby(["center_id", "week"])["id"].agg(['size']).reset_index()
gdf.columns = ["center_id", "week", "center_week_count"]
total_data = pd.merge(total_data,gdf, on=["center_id", "week"], how="left")

gdf = total_data.groupby(["center_id", "category"])["id"].count().reset_index()
gdf.columns = ["center_id", "category", "center_cat_count"]
total_data = pd.merge(total_data,gdf, on=["center_id", "category"], how="left")

gdf = total_data.groupby(["center_id", "category", "week"])["id"].count().reset_index()
gdf.columns = ["center_id", "category", "week", "center_cat_week_count"]
total_data = pd.merge(total_data,gdf, on=["center_id", "category", "week"], how="left")

gdf = total_data.groupby(["center_id", "cuisine"])["id"].count().reset_index()
gdf.columns = ["center_id", "cuisine", "center_cui_count"]
total_data = pd.merge(total_data,gdf, on=["center_id", "cuisine"], how="left")

In [39]:
### Meal count features ###
gdf = total_data.groupby(["meal_id"])["id"].count().reset_index()
gdf.columns = ["meal_id", "meal_count"]
total_data = pd.merge(total_data,gdf, on=["meal_id"], how="left")

gdf = total_data.groupby(["region_code", "meal_id"])["id"].count().reset_index()
gdf.columns = ["region_code", "meal_id", "region_meal_count"]
total_data= pd.merge(total_data,gdf, on=["region_code", "meal_id"], how="left")

gdf = total_data.groupby(["meal_id", "week"])["id"].count().reset_index()
gdf.columns = ["meal_id", "week", "meal_week_count"]
total_data = pd.merge(total_data,gdf, on=["meal_id", "week"], how="left")

gdf =total_data.groupby(["center_type", "meal_id", "week"])["id"].count().reset_index()
gdf.columns = ["center_type", "meal_id", "week", "type_meal_week_count"]
total_data = pd.merge(total_data,gdf, on=["center_type", "meal_id", "week"], how="left")

gdf = total_data.groupby(["region_code", "meal_id", "week"])["id"].count().reset_index()
gdf.columns = ["region_code", "meal_id", "week", "region_meal_week_count"]
total_data = pd.merge(total_data,gdf, on=["region_code", "meal_id", "week"], how="left")

gdf =total_data.groupby(["city_code", "meal_id", "week"])["id"].count().reset_index()
gdf.columns = ["city_code", "meal_id", "week", "city_meal_week_count"]
total_data = pd.merge(total_data,gdf, on=["city_code", "meal_id", "week"], how="left")

In [40]:
### Price rank ###
total_data["meal_price_rank"] = total_data.groupby("meal_id")["checkout_price"].rank()
total_data["meal_city_price_rank"] =total_data.groupby(["meal_id", "city_code"])["checkout_price"].rank()
total_data["meal_region_price_rank"] = total_data.groupby(["meal_id", "region_code"])["checkout_price"].rank()
total_data["meal_week_price_rank"] = total_data.groupby(["meal_id", "week"])["checkout_price"].rank()

total_data["center_price_rank"] = total_data.groupby("center_id")["checkout_price"].rank()
total_data["center_week_price_rank"] = total_data.groupby(["center_id", "week"])["checkout_price"].rank()
total_data["center_cat_price_rank"] = total_data.groupby(["center_id", "category"])["checkout_price"].rank()

In [41]:
### Week features ###
gdf = total_data.groupby(["meal_id"])["checkout_price"].agg(["min", "max", "mean", "std"]).reset_index()
gdf.columns = ["meal_id", "meal_price_min", "meal_price_max", "meal_price_mean", "meal_price_std"]
total_data = pd.merge(total_data,gdf, on=["meal_id"], how="left")

gdf = total_data.groupby(["meal_id"])["base_price"].agg(["min", "max", "mean", "std"]).reset_index()
gdf.columns = ["meal_id", "disc_price_min", "disc_price_max", "disc_price_mean", "disc_price_std"]
total_data = pd.merge(total_data,gdf, on=["meal_id"], how="left")

gdf = total_data.groupby(["city_code","meal_id", "week"])["checkout_price"].agg(["min", "max", "mean", "std"]).reset_index()
gdf.columns = ["city_code", "meal_id", "week", "meal_price2_min", "meal_price2_max", "meal_price2_mean", "meal_price2_std"]
total_data= pd.merge(total_data,gdf, on=["city_code", "meal_id", "week"], how="left")

gdf = total_data.groupby(["city_code", "category"])["checkout_price"].agg(["mean", "std"]).reset_index()
gdf.columns = ["city_code", "category", "meal_price3_mean", "meal_price3_std"]
total_datal = pd.merge(total_data,gdf, on=["city_code", "category"], how="left")

In [43]:
total_data['center_id'] = total_data['center_id'].astype(object)
total_data['meal_id'] = total_data['meal_id'].astype(object)
total_data['region_code'] = total_data['region_code'].astype(object)
total_data['city_code'] = total_data['city_code'].astype(object)


In [44]:
train_data = total_data.loc[total_data['train_or_test']=='train', :]
test_data = total_data.loc[total_data['train_or_test']=='test', :]

In [45]:
center_meal_mean = train_data.groupby(['center_id','meal_id'])['num_orders'].mean()
center_meal_mean = center_meal_mean.reset_index()
center_meal_mean = center_meal_mean.rename(columns={'num_orders':'avg_orders'})
train_data = pd.merge(train_data,center_meal_mean, how='left', left_on=['center_id','meal_id'], right_on = ['center_id','meal_id'])
test_data = pd.merge(test_data,center_meal_mean, how='left', left_on=['center_id','meal_id'], right_on = ['center_id','meal_id'])
center_mean = train_data.groupby('center_id')['num_orders'].mean()
center_mean = center_mean.reset_index()
center_mean = center_mean.rename(columns={'num_orders':'avg_center'})
train_data = pd.merge(train_data,center_mean, how='left', left_on=['center_id'], right_on = ['center_id'])
test_data = pd.merge(test_data,center_mean, how='left', left_on=['center_id'], right_on = ['center_id'])
meal_mean = train_data.groupby('meal_id')['num_orders'].mean()
meal_mean = meal_mean.reset_index()
meal_mean = meal_mean.rename(columns={'num_orders':'avg_meal'})
train_data = pd.merge(train_data, meal_mean,how='left',left_on=['meal_id'], right_on = ['meal_id'])
test_data = pd.merge(test_data, meal_mean,how='left',left_on=['meal_id'], right_on = ['meal_id'])

In [46]:
train_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 456546 entries, 0 to 456545
Data columns (total 60 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   id                         456546 non-null  int64  
 1   week                       456546 non-null  int64  
 2   center_id                  456546 non-null  object 
 3   meal_id                    456546 non-null  object 
 4   checkout_price             456546 non-null  float64
 5   base_price                 456546 non-null  float64
 6   emailer_for_promotion      456546 non-null  int64  
 7   homepage_featured          456546 non-null  int64  
 8   num_orders                 456546 non-null  float64
 9   city_code                  456546 non-null  object 
 10  region_code                456546 non-null  object 
 11  center_type                456546 non-null  object 
 12  op_area                    456546 non-null  float64
 13  category                   45

In [47]:
test_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32573 entries, 0 to 32572
Data columns (total 60 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         32573 non-null  int64  
 1   week                       32573 non-null  int64  
 2   center_id                  32573 non-null  object 
 3   meal_id                    32573 non-null  object 
 4   checkout_price             32573 non-null  float64
 5   base_price                 32573 non-null  float64
 6   emailer_for_promotion      32573 non-null  int64  
 7   homepage_featured          32573 non-null  int64  
 8   num_orders                 0 non-null      float64
 9   city_code                  32573 non-null  object 
 10  region_code                32573 non-null  object 
 11  center_type                32573 non-null  object 
 12  op_area                    32573 non-null  float64
 13  category                   32573 non-null  obj

In [ ]:
#Catboost Model


In [48]:
avoid_column = ['id', 'num_orders', 'train_or_test']


In [49]:
features = [col for col in test_data.columns if col not in avoid_column]


In [50]:
features


['week',
 'center_id',
 'meal_id',
 'checkout_price',
 'base_price',
 'emailer_for_promotion',
 'homepage_featured',
 'city_code',
 'region_code',
 'center_type',
 'op_area',
 'category',
 'cuisine',
 'discount_on_base',
 'discount_ratio',
 'price_last_curr_diff',
 'num_orders_lag_10',
 'num_orders_lag_11',
 'num_orders_lag_12',
 'num_orders_lag_10_ewm_0.5',
 'num_orders_lag_11_ewm_0.5',
 'num_orders_lag_12_ewm_0.5',
 'num_orders_lag_13_ewm_0.5',
 'num_orders_lag_14_ewm_0.5',
 'num_orders_lag_15_ewm_0.5',
 'center_week_count',
 'center_cat_count',
 'center_cat_week_count',
 'center_cui_count',
 'meal_count',
 'region_meal_count',
 'meal_week_count',
 'type_meal_week_count',
 'region_meal_week_count',
 'city_meal_week_count',
 'meal_price_rank',
 'meal_city_price_rank',
 'meal_region_price_rank',
 'meal_week_price_rank',
 'center_price_rank',
 'center_week_price_rank',
 'center_cat_price_rank',
 'meal_price_min',
 'meal_price_max',
 'meal_price_mean',
 'meal_price_std',
 'disc_price_min

In [51]:
categorical_features_indices = np.where(train_data[features].dtypes =='object')[0]
categorical_features_indices

array([ 1,  2,  7,  8,  9, 11, 12], dtype=int64)

In [52]:
model = CatBoostRegressor(
    iterations=2000, 
    learning_rate=0.02, 
    max_depth=8, 
    l2_leaf_reg=10, 
    loss_function='RMSE',
    random_seed=2019,
    od_type='Iter',
    od_wait=25,
    verbose=100,
    use_best_model=True
    )

In [53]:
# แบ่งข้อมูลเป็นชุดทดสอบและชุดการทดสอบโดยใช้ KFold

errcb=[]
y_pred_test=[]
# แบ่งข้อมูลเป็นชุดทดสอบและชุดการทดสอบโดยใช้ KFold

fold = KFold(n_splits=5,shuffle=True,random_state=2019)

for train_index, test_index in fold.split(train_data[features],train_data['num_orders']):
    X_train, X_valid =train_data[features].iloc[train_index], train_data[features].iloc[test_index]
    y_train, y_valid = train_data['num_orders'][train_index],train_data['num_orders'][test_index]
    # ฝึกโมเดลบนชุดฝึกและประเมินบนชุดการทดสอบ

    model.fit(X_train,y_train,
              cat_features = categorical_features_indices,
              eval_set=(X_valid,y_valid),
              early_stopping_rounds=300,
              verbose=100)
# ทำนายค่าบนชุดการทดสอบและคำนวณค่าคลาดเคลื่อน

    preds = model.predict(X_valid)
    print("err: ",np.sqrt(mean_squared_error(y_valid,preds)))
    errcb.append(np.sqrt(mean_squared_error(y_valid,preds)))
    p = model.predict(test_data[features])
    y_pred_test.append(p)

0:	learn: 1.1849585	test: 1.1842571	best: 1.1842571 (0)	total: 1.32s	remaining: 43m 58s
100:	learn: 0.5389654	test: 0.5407121	best: 0.5407121 (100)	total: 1m 28s	remaining: 27m 35s
200:	learn: 0.4974690	test: 0.4997800	best: 0.4997800 (200)	total: 3m 3s	remaining: 27m 20s
300:	learn: 0.4868282	test: 0.4895089	best: 0.4895089 (300)	total: 4m 48s	remaining: 27m 10s
400:	learn: 0.4803016	test: 0.4833723	best: 0.4833723 (400)	total: 6m 19s	remaining: 25m 13s
500:	learn: 0.4757436	test: 0.4792053	best: 0.4792053 (500)	total: 7m 46s	remaining: 23m 16s
600:	learn: 0.4718133	test: 0.4756793	best: 0.4756793 (600)	total: 9m 11s	remaining: 21m 24s
700:	learn: 0.4684981	test: 0.4727433	best: 0.4727433 (700)	total: 10m 42s	remaining: 19m 50s
800:	learn: 0.4655701	test: 0.4702085	best: 0.4702085 (800)	total: 12m	remaining: 17m 58s
900:	learn: 0.4631435	test: 0.4681716	best: 0.4681716 (900)	total: 13m 16s	remaining: 16m 11s
1000:	learn: 0.4611101	test: 0.4664191	best: 0.4664191 (1000)	total: 14m 29s	

In [55]:
model.fit(
    X=train_data[features],
    y=train_data['num_orders'],
    cat_features=categorical_features_indices,
    verbose=100,
    eval_set=(X_valid, y_valid)  # Replace X_valid and y_valid with your validation data
)


0:	learn: 1.1850418	test: 1.1842361	best: 1.1842361 (0)	total: 705ms	remaining: 23m 28s
100:	learn: 0.5387679	test: 0.5371396	best: 0.5371396 (100)	total: 1m 33s	remaining: 29m 20s
200:	learn: 0.4976165	test: 0.4961929	best: 0.4961929 (200)	total: 3m 18s	remaining: 29m 36s
300:	learn: 0.4873530	test: 0.4860074	best: 0.4860074 (300)	total: 5m 28s	remaining: 30m 57s
400:	learn: 0.4807765	test: 0.4794632	best: 0.4794632 (400)	total: 7m 43s	remaining: 30m 46s
500:	learn: 0.4762269	test: 0.4748946	best: 0.4748946 (500)	total: 10m 11s	remaining: 30m 30s
600:	learn: 0.4722575	test: 0.4709310	best: 0.4709310 (600)	total: 12m 27s	remaining: 29m
700:	learn: 0.4690898	test: 0.4677610	best: 0.4677610 (700)	total: 14m 32s	remaining: 26m 56s
800:	learn: 0.4662494	test: 0.4649379	best: 0.4649379 (800)	total: 16m 35s	remaining: 24m 49s
900:	learn: 0.4638107	test: 0.4624972	best: 0.4624972 (900)	total: 18m 37s	remaining: 22m 42s
1000:	learn: 0.4618411	test: 0.4605356	best: 0.4605356 (1000)	total: 20m 3